# TESTING NS FOR LINE FITTING

In [1]:
import numpyro as npro
npro.set_host_device_count(4)
import jax
jax.local_device_count()
import jax.numpy as jnp
import tqdm
import blackjax
import ravel
import pandas as pd
import os
from glob import glob
import matplotlib.pyplot as plt

JAX 64-bit enabled: False


In [2]:
from ravel import read_spectra, setup_star_directory_and_save_jds, setup_line_dictionary, initialize_fit_variables, rv_shift_wavelength
from ravel import gaussian, lorentzian, pseudo_voigt
import numpy as np
from scipy.interpolate import interp1d
from astropy.constants import c



In [14]:
# RAVEL.PY WORK ON fit_sb2_probmod
from tensorflow_probability.substrates import jax as tfp
import distrax


def fit_sb2_probmod(lines, wavelengths, fluxes, f_errors, lines_dic, Hlines, neblines, path, sigma_prior, K=2, shift_kms=0,
                    wavelength_type='air', rm_epochs=None, profile='Voigt', chi2_plots=False):
    """
    Fit SB2 (double-lined spectroscopic binary) spectral lines using a probabilistic
    model with Numpyro. The function interpolates spectral data onto a common grid,
    constructs a Bayesian model for the line profiles, and samples the posterior via
    MCMC (using NUTS). To improve robustness against switching, a second model is constructed
    that re-samples only the RV posteriors. The best-fitting result between both runs is 
    determined via a χ2 comparison.
    
    Parameters:
    -----------
    lines : list
        List of spectral line identifiers (keys from lines_dic) to be fitted.
    wavelengths : list
        List (per epoch) of wavelength arrays.
    fluxes : list
        List (per epoch) of flux arrays.
    f_errors : list
        List (per epoch) of flux error arrays.
    lines_dic : dict
        Dictionary containing spectral line regions, initial centre guesses, etc.
    Hlines : list
        List of lines (subset of `lines`) that are Hydrogen lines.
    neblines : list
        (Currently unused) List of nebular lines.
    path : str
        Path for storing output plots.
    sigma_prior : float
        Sigma on Gaussian priors for RV fitting in the second MCMC run [km/s].
    K : int, optional
        Number of components (default 2).
    shift_kms : float, optional
        The overall velocity shift in km/s. For example, use 172 km/s for the SMC.
    wavelength_type : str, optional
        Type of wavelength to use ('air' or 'vacuum'). Default is 'air'.
    rm_epochs : list, optional
        The indices (0-based) of the epochs to remove from the fitting.
    profile : str, optional ('Voigt' or 'Gaussian)
        Profile type used for all components in the spectral line fitting.
    chi2_plots : bool
        Enable/disable seperate plotting of first, second, and final (stitched) MCMC results.

    Returns:
    --------
    trace : dict
        The MCMC trace (posterior samples).
    x_waves : array (JAX)
        The interpolated wavelength grid for each line and epoch.
    y_fluxes : array (JAX)
        The interpolated fluxes.
    """
    # --- basics ---
    n_lines = len(lines)
    n_epochs = len(wavelengths)
    print('Number of lines:', n_lines)
    print('Number of epochs:', n_epochs)

    key = 'centre' if wavelength_type == 'vacuum' else 'air'
    is_hline = jnp.array([line in Hlines for line in lines])

    # --- interpolate to common grid ---
    x_waves_interp, y_fluxes_interp, y_errors_interp = [], [], []
    common_grid_length = 200

    for line in lines:
        region_start, region_end = lines_dic[line]['region']
        region_start = rv_shift_wavelength(region_start, shift_kms)
        region_end   = rv_shift_wavelength(region_end,   shift_kms)

        x_waves_line, y_fluxes_line, y_errors_line = [], [], []

        for wave_set, flux_set, error_set in zip(wavelengths, fluxes, f_errors):
            mask = (wave_set > region_start) & (wave_set < region_end)
            wave_masked = wave_set[mask]
            flux_masked = flux_set[mask]
            if error_set is not None:
                error_masked = error_set[mask]
            else:
                f_err = compute_flux_err(wave_set, flux_set)
                error_masked = f_err[mask]

            common_wavelength_grid = np.linspace(wave_masked.min(), wave_masked.max(), common_grid_length)
            interp_flux  = interp1d(wave_masked, flux_masked,  bounds_error=False, fill_value="extrapolate")(common_wavelength_grid)
            interp_error = interp1d(wave_masked, error_masked, bounds_error=False, fill_value="extrapolate")(common_wavelength_grid)
            x_waves_line.append(common_wavelength_grid)
            y_fluxes_line.append(interp_flux)
            y_errors_line.append(interp_error)

        x_waves_interp.append(x_waves_line)
        y_fluxes_interp.append(y_fluxes_line)
        y_errors_interp.append(y_errors_line)

    x_waves  = jnp.array(x_waves_interp)   # (L, E, N)
    y_fluxes = jnp.array(y_fluxes_interp)  # (L, E, N)
    y_errors = jnp.array(y_errors_interp)  # (L, E, N)

    if rm_epochs is not None:
        x_waves  = jnp.delete(x_waves,  jnp.array(rm_epochs), axis=1)
        y_fluxes = jnp.delete(y_fluxes, jnp.array(rm_epochs), axis=1)
        y_errors = jnp.delete(y_errors, jnp.array(rm_epochs), axis=1)

    cen_ini = jnp.array([lines_dic[line][key][0] for line in lines])

    # --- priors ---
    c_kms = c.to('km/s').value
    nlines, nepochs, ndata = x_waves.shape

    tfd = tfp.distributions

    logσ_ε_prior = tfd.Uniform(low=-5.0, high=0.0)
    def make_eps_prior_tfp(logσ_ε):
        return tfd.TruncatedNormal(loc=1.0, scale=jnp.exp(logσ_ε), low=0.7, high=1.1)

    λ_rest_prior = distrax.Independent(
        tfd.Normal(loc=cen_ini, scale=jnp.full(nlines, 1e-3)),
        reinterpreted_batch_ndims=1
    )

    amp0_prior = tfd.Independent(
        tfd.TruncatedNormal(loc=jnp.full(nlines, 0.18),
                            scale=jnp.full(nlines, 0.06),
                            low=jnp.full(nlines, 0.02),
                            high=jnp.full(nlines, 0.40)),
        1
    )
    amp_ratio_prior = tfd.Independent(
        tfd.TruncatedNormal(loc=jnp.full(nlines, 0.60),
                            scale=jnp.full(nlines, 0.15),
                            low=jnp.full(nlines, 0.25),
                            high=jnp.full(nlines, 0.95)),
        1
    )

    wid_G1_prior      = distrax.Independent(distrax.Uniform(low=jnp.full(nlines, 0.5), high=jnp.full(nlines, 5.0)), 1)
    wid_L1_prior      = distrax.Independent(distrax.Uniform(low=jnp.full(nlines, 0.1), high=jnp.full(nlines, 3.0)), 1)
    delta_wid_G_prior = distrax.Independent(distrax.Uniform(low=jnp.full(nlines, 0.1), high=jnp.full(nlines, 2.0)), 1)
    delta_wid_L_prior = distrax.Independent(distrax.Uniform(low=jnp.full(nlines, 0.05), high=jnp.full(nlines, 1.0)), 1)

    σ_Δv = 200.
    comp_sep = 200.
    Δv_means = jnp.array([shift_kms - comp_sep/2, shift_kms + comp_sep/2])  # (K,)
    Δv_loc   = jnp.tile(Δv_means[:, None], (1, nepochs))                    # (K,E)
    Δv_τk_prior = distrax.Independent(distrax.Normal(loc=Δv_loc, scale=jnp.full((K, nepochs), 200.0)), 2)

    def prior_sample(seed, sample_shape=()):
        k = jax.random.split(seed, 10)
        logσ_ε   = logσ_ε_prior.sample(seed=k[0], sample_shape=sample_shape)     # (B,)
        ε        = make_eps_prior_tfp(logσ_ε).sample(seed=k[1])                  # (B,)  <-- no extra sample_shape
        amp0     = amp0_prior.sample(seed=k[2], sample_shape=sample_shape)       # (B,L) or (L,)
        amp_ratio= amp_ratio_prior.sample(seed=k[3], sample_shape=sample_shape)
        λ_rest   = λ_rest_prior.sample(seed=k[4], sample_shape=sample_shape)
        wid_G1   = wid_G1_prior.sample(seed=k[5], sample_shape=sample_shape)
        wid_L1   = wid_L1_prior.sample(seed=k[6], sample_shape=sample_shape)
        delta_wid_G = delta_wid_G_prior.sample(seed=k[7], sample_shape=sample_shape)
        delta_wid_L = delta_wid_L_prior.sample(seed=k[8], sample_shape=sample_shape)
        Δv_τk    = Δv_τk_prior.sample(seed=k[9], sample_shape=sample_shape)
        return {
            "logσ_ε": logσ_ε, "ε": ε, "λ_rest": λ_rest,
            "amp0": amp0, "amp_ratio": amp_ratio,
            "wid_G1": wid_G1, "wid_L1": wid_L1,
            "delta_wid_G": delta_wid_G, "delta_wid_L": delta_wid_L,
            "Δv_τk": Δv_τk,
        }

    def prior_log_prob(params):
        lp  = logσ_ε_prior.log_prob(params["logσ_ε"])
        lp += make_eps_prior_tfp(params["logσ_ε"]).log_prob(params["ε"])
        lp += amp0_prior.log_prob(params["amp0"])
        lp += amp_ratio_prior.log_prob(params["amp_ratio"])
        lp += λ_rest_prior.log_prob(params["λ_rest"])
        lp += wid_G1_prior.log_prob(params["wid_G1"])
        lp += wid_L1_prior.log_prob(params["wid_L1"])
        lp += delta_wid_G_prior.log_prob(params["delta_wid_G"])
        lp += delta_wid_L_prior.log_prob(params["delta_wid_L"])
        lp += Δv_τk_prior.log_prob(params["Δv_τk"])
        return lp

    def flux_pred(params, lam_grid, is_hline, profile="Voigt"):
        """
        Unbatched: expects single-particle leaves:
          ε: (), λ_rest: (L,), amp0: (L,), amp_ratio: (L,),
          wid_G1: (L,), wid_L1: (L,), delta_wid_G: (L,), delta_wid_L: (L,),
          Δv_τk: (K,E); lam_grid: (L,E,N); is_hline: (L,)
        Returns f_pred: (L,E,N)
        """
        L, E, N = lam_grid.shape
        ε          = params["ε"]
        amp0       = params["amp0"]
        amp_ratio  = params["amp_ratio"]
        λ_rest     = params["λ_rest"]
        wid_G1     = params["wid_G1"]
        wid_L1     = params["wid_L1"]
        delta_wid_G= params["delta_wid_G"]
        delta_wid_L= params["delta_wid_L"]
        dv_tau     = params["Δv_τk"]     # (K,E)

        # components (K=2)
        amp1 = amp_ratio * amp0
        A  = jnp.stack([amp0, amp1], axis=0)[:, :, None, None]                 # (K,L,1,1)
        wG = jnp.stack([wid_G1, wid_G1 + delta_wid_G], axis=0)[:, :, None, None]
        wL = jnp.stack([wid_L1, wid_L1 + delta_wid_L], axis=0)[:, :, None, None]

        lam0 = λ_rest[None, :, None, None]                                     # (1,L,1,1)
        dv   = dv_tau[:, None, :, None]                                        # (K,1,E,1)
        mu   = lam0 * (1.0 + dv / c_kms)                                       # (K,L,E,1)

        lam  = lam_grid[None, :, :, :]                                         # (1,L,E,N)
        isH  = is_hline[None, :, None, None]                                   # (1,L,1,1)

        G  = gaussian(lam, A, mu, wG)                                          # (K,L,E,N)
        Lr = lorentzian(lam, A, mu, wL)                                        # (K,L,E,N)
        V  = pseudo_voigt(lam, A, mu, wG, wL)                                  # (K,L,E,N)

        comp = jnp.where(isH, Lr, (V if profile == "Voigt" else G))            # (K,L,E,N)
        return ε + comp.sum(axis=0)                                            # (L,E,N)

    def loglikelihood_fn(params):
        f_pred = flux_pred(params, x_waves, is_hline, profile)                  # (L,E,N)
        resid  = (y_fluxes - f_pred) / y_errors
        log_like = -jnp.log(y_errors * jnp.sqrt(2*jnp.pi)) - 0.5 * resid**2
        return jnp.sum(log_like)                                                # scalar

    print(f"\nFitting with profile: {profile}, Δv_means: {Δv_means}")

    # NS from Wk1 notebook
    rng_key  = jax.random.PRNGKey(0)
    n_live   = 1200
    initial_population = prior_sample(rng_key, sample_shape=(n_live,))

    d_guess    = (1 + 1) + 3*nlines + 4*nlines + (K*nepochs)
    num_delete = int(n_live * 0.1)

    algo = blackjax.nss(
        logprior_fn=prior_log_prob,
        loglikelihood_fn=loglikelihood_fn,
        num_delete=num_delete,
        num_inner_steps=3 * d_guess,
    )

    state = algo.init(initial_population)

    # --- NS loop ---
    @jax.jit
    def one_step(carry, xs):
        state, k = carry
        k, subk = jax.random.split(k, 2)
        state, dead_point = algo.step(subk, state)
        return (state, k), dead_point

    iteration_count = 0
    live_points_snapshots = []
    dead = []

    tol = 230.0  # stop when (logZ_live - logZ) < tol
    with tqdm.tqdm(desc="Dead points", unit=" dead points") as pbar:
        live_points_snapshots.append(state.particles["Δv_τk"])  # (n_live, K, E)
        while not (state.logZ_live - state.logZ < tol):
            print('iteration:', iteration_count,
                  ' logZ_live:', float(state.logZ_live),
                  ' logZ:', float(state.logZ),
                  flush=True)
            iteration_count += 1
            (state, rng_key), dead_info = one_step((state, rng_key), None)
            dead.append(dead_info)
            pbar.update(num_delete)
            live_points_snapshots.append(state.particles["Δv_τk"])

    live_points_snapshots.append(state.particles["Δv_τk"])

    # --- post-processing ---
    from blackjax.ns.utils import log_weights, finalise, sample, ess
    rng_key, weight_key, sample_key = jax.random.split(rng_key, 3)

    final_state = finalise(state, dead)
    log_w   = log_weights(weight_key, final_state, shape=100)
    samples = sample(sample_key, final_state, shape=n_live)
    ns_ess  = ess(sample_key, final_state)
    logzs   = jax.scipy.special.logsumexp(log_w, axis=0)

    # posterior draws (same as samples above, keeping your variable name)
    rng_key, sk = jax.random.split(rng_key)
    posterior = sample(sk, final_state, shape=n_live)

    return final_state, posterior, live_points_snapshots

In [15]:
sys_path = 'SB2_sys019_snr30/'
spec_files = sorted(glob(os.path.join(sys_path, "*epoch??.txt")))
SB2 = True

out_dir_requested = os.path.join('Sample_fit_ns', sys_path)
os.makedirs(out_dir_requested, exist_ok=True)

Hlines = [4102, 4340, 4861, 6562]
print('*** SB2 set to:', 'True', '***\n')

# Read in spectral data from the provided file list and data path
wavelengths, fluxes, f_errors, names, jds = read_spectra(spec_files, sys_path, 'txt', SB2=True)

# Setup the output directory and save the JD information if available
out_path = setup_star_directory_and_save_jds(names, jds, sys_path, SB2)

# Get the dictionary with line regions and initial parameters
lines = [4026, 4144, 4388, 4471]
lines_dic = setup_line_dictionary()

# Verify that user‑requested lines exist in the dictionary
missing = [ln for ln in lines if ln not in lines_dic]
if missing:
    print("Error: Unknown spectral line identifier(s):", missing)
    print("Available lines are:", sorted(lines_dic.keys()))
    raise ValueError(f"Please choose from the available lines or add your own. Missing: {missing}")

print('\n*** Fitting lines ***')
print('---------------------')
print('Lines to be fitted:', lines)

# Initialize fit variables for each line (lists to hold fit results, uncertainties, etc.)
(cen1, cen1_er, amp1, amp1_er, wid1, wid1_er, 
    cen2, cen2_er, amp2, amp2_er, wid2, wid2_er, 
    dely, sdev, results, comps, delta_cen, chisqr) = initialize_fit_variables(lines)


# SB2 fitting: fit all lines using the probabilistic SB2 model and write results to CSV
# final_state, posterior, live_points_snapshots = fit_sb2_probmod(lines, wavelengths, fluxes, f_errors, lines_dic,
#                                             Hlines, neblines=[], out_path, K=2, shift_kms=0)

final_state, posterior, live_points_snapshots = fit_sb2_probmod(lines, wavelengths, fluxes, f_errors, lines_dic, Hlines, neblines=1, path=1, sigma_prior=1, K=2, shift_kms=0,
                    wavelength_type='air', rm_epochs=None, profile='Voigt', chi2_plots=False)


*** SB2 set to: True ***


*** Fitting lines ***
---------------------
Lines to be fitted: [4026, 4144, 4388, 4471]
Number of lines: 4
Number of epochs: 12

Fitting with profile: Voigt, Δv_means: [-100.  100.]


Dead points: 0 dead points [00:00, ? dead points/s]

iteration: 0  logZ_live: 15542.90625  logZ: -inf


Dead points: 120 dead points [00:40,  2.93 dead points/s]

iteration: 1  logZ_live: 15542.80078125  logZ: -53948.5546875


...

Dead points: 29760 dead points [52:13, 10.80 dead points/s]

iteration: 248  logZ_live: 17839.791015625  logZ: 17603.15234375


Dead points: 29880 dead points [52:23, 11.28 dead points/s]

iteration: 249  logZ_live: 17840.7578125  logZ: 17608.4765625


Dead points: 30000 dead points [52:45,  9.48 dead points/s]


In [30]:
import numpy as np
import matplotlib.pyplot as plt
import imageio
import os

def make_live_scatter_gif(live_points_snapshots, epoch=0, out='live_scatter_epoch{:02d}.gif',
                          xlim=None, ylim=None, step=1, duration=0.4, alpha=0.5, dpi=120):
    """
    live_points_snapshots: list of arrays with shape (n_live, K, E) for each iteration
    epoch: which epoch index to visualize
    step: use every `step`-th snapshot to keep the gif light
    """
    out = out.format(epoch)
    frames = []
    tmp_png = '_tmp_frame.png'

    for it, snap in enumerate(live_points_snapshots[::step]):
        arr = np.asarray(snap)                      # (n_live, K, E)
        v1 = arr[:, 0, epoch]
        v2 = arr[:, 1, epoch]

        plt.figure(figsize=(6,6), dpi=dpi)
        plt.xlim([-810,810])
        plt.ylim([-810,810])
        plt.scatter(v1, v2, s=10, alpha=alpha, c='cornflowerblue')
        plt.xlabel(r'$v_1\ \mathrm{(km\,s^{-1})}$')
        plt.ylabel(r'$v_2\ \mathrm{(km\,s^{-1})}$')
        plt.title(f'Live points @ epoch: {epoch}, iteration_no: {it*step})')
        plt.scatter([-22,22], [-22,22], marker='x', c='r', s=20)
        plt.gca().set_aspect('equal', adjustable='box')
        plt.tight_layout()
        plt.savefig(tmp_png)
        plt.close()
        frames.append(imageio.imread(tmp_png))

    imageio.mimsave(out, frames, duration=duration)
    try: os.remove(tmp_png)
    except OSError: pass
    print(f'Wrote {out}')


In [31]:
# Example: animate epoch 0, every 2nd snapshot
make_live_scatter_gif(live_points_snapshots, epoch=1, step=2, duration=0.3)


/var/folders/l8/g2bx224n7q35swctm351kzqc0000gn/T/ipykernel_32617/1107619448.py:42: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  frames.append(imageio.imread(tmp_png))


Wrote live_scatter_epoch01.gif
